In [1]:
import numpy as np
from scipy import stats
import time
import datetime
import pandas as pd

import mysql.connector
from mysql.connector import Error

from bokeh.plotting import *
from bokeh.layouts import gridplot
from bokeh.models import *# Span, ColumnDataSource, LogColorMapper, ColorMapper, LogTicker, ColorBar, BasicTicker, LinearColorMapper, PrintfTickFormatter, HoverTool, CategoricalColorMapper, Range1d, Title
from bokeh.models.widgets import Tabs, Panel
from bokeh.io import show, output_notebook, reset_output
output_notebook()
from bokeh.models.glyphs import Text
import bokeh.palettes as bp
from bokeh.transform import factor_cmap

# import hail as hl
import json
import urllib
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)

from pathlib import Path
from joblib import Parallel, delayed

Loading BokehJS ...

In [2]:
json_url = urllib.request.urlopen('https://api.helioviewer.org/?action=getDataSources')
hv_keys = json.loads(json_url.read())

In [3]:
hv={}

try:
    connection = mysql.connector.connect(host='localhost',
                                         database='hv',
                                         user='hv_varun',
                                         password='Helioviewer@2020')

#     sql_select_Query = "SELECT date_format(date, '%Y-%m-%d 00:00:00') as date, count(*) as count FROM data FORCE INDEX (date_index) WHERE sourceId={} GROUP BY date_format(date, '%Y-%m-%d 00:00:00');".format(sourceId)
#     sql_select_Query = "SELECT filepath, date, sourceid FROM data WHERE sourceId=%d LIMIT 20;" %sourceId
#     sql_select_Query = "SELECT sourceId, date_format(date, '%Y-%m-%d 00:00:00') as date, count(*) as count FROM data FORCE INDEX (date_index) WHERE sourceId=8 GROUP BY date_format(date, '%Y-%m-%d 00:00:00'), sourceId;"
    sql_select_Query = "SELECT date_format(timestamp, '%Y-%m-%d 00:00:00') as date, count(*) as count FROM movies GROUP BY date_format(timestamp, '%Y-%m-%d 00:00:00');"#.format(sourceId)
#     sql_select_Query = "SELECT date_format(date, '%Y-%m-%d 00:00:00') as date, count(*) as count FROM data FORCE INDEX (date_index) GROUP BY date_format(date, '%Y-%m-%d 00:00:00');"
#     sql_select_Query = "SELECT count(*) FROM data WHERE filepath LIKE '/AIA/1600/%';"
#     sql_select_Query = "SELECT * FROM movies LIMIT 20;"
    cursor = connection.cursor()
    cursor.execute(sql_select_Query)
    records = cursor.fetchall()
    hv['movies'] = pd.DataFrame(records, columns=cursor.column_names)
except Error as e:
    print("Error reading data from MySQL table", e)
finally:
    if (connection.is_connected()):
        connection.close()
        cursor.close()
print("movies done")

try:
    connection = mysql.connector.connect(host='localhost',
                                         database='hv',
                                         user='hv_varun',
                                         password='Helioviewer@2020')

#     sql_select_Query = "SELECT date_format(date, '%Y-%m-%d 00:00:00') as date, count(*) as count FROM data FORCE INDEX (date_index) WHERE sourceId={} GROUP BY date_format(date, '%Y-%m-%d 00:00:00');".format(sourceId)
#     sql_select_Query = "SELECT filepath, date, sourceid FROM data WHERE sourceId=%d LIMIT 20;" %sourceId
#     sql_select_Query = "SELECT sourceId, date_format(date, '%Y-%m-%d 00:00:00') as date, count(*) as count FROM data FORCE INDEX (date_index) WHERE sourceId=8 GROUP BY date_format(date, '%Y-%m-%d 00:00:00'), sourceId;"
    sql_select_Query = "SELECT date_format(timestamp, '%Y-%m-%d 00:00:00') as date, count(*) as count FROM screenshots GROUP BY date_format(timestamp, '%Y-%m-%d 00:00:00');"#.format(sourceId)#     sql_select_Query = "SELECT date_format(date, '%Y-%m-%d 00:00:00') as date, count(*) as count FROM data FORCE INDEX (date_index) GROUP BY date_format(date, '%Y-%m-%d 00:00:00');"
#     sql_select_Query = "SELECT count(*) FROM data WHERE filepath LIKE '/AIA/1600/%';"
#     sql_select_Query = "SELECT * FROM screenshots LIMIT 20;"
    cursor = connection.cursor()
    cursor.execute(sql_select_Query)
    records = cursor.fetchall()
    hv['screenshots'] = pd.DataFrame(records, columns=cursor.column_names)
except Error as e:
    print("Error reading data from MySQL table", e)
finally:
    if (connection.is_connected()):
        connection.close()
        cursor.close()
print("screenshots done")

movies done
screenshots done


In [4]:
hv['movies']['date'] = pd.to_datetime(hv['movies']['date'])
hv['movies'] = hv['movies'].sort_values(['date']).reset_index(drop=True)
hv['movies']

,date,count
0,2011-05-26,40
1,2011-05-27,146
2,2011-05-28,205
3,2011-05-29,224
4,2011-05-30,204
...,...,...
3046,2020-08-20,95
3047,2020-08-21,168
3048,2020-08-22,102
3049,2020-08-23,64


In [6]:
hv['screenshots']['date'] = pd.to_datetime(hv['screenshots']['date'])
hv['screenshots'] = hv['screenshots'].sort_values(['date']).reset_index(drop=True)
hv['screenshots']

,date,count
0,2011-05-26,6
1,2011-05-27,16
2,2011-05-28,18
3,2011-05-29,45
4,2011-05-30,67
...,...,...
3049,2020-08-20,13404
3050,2020-08-21,12252
3051,2020-08-22,10435
3052,2020-08-23,10490


# Time series

In [14]:
print("{:,}".format(df['count'].sum()))

21,993,691


In [17]:
for key in hv.keys():
    df = hv[key].copy()
    df = df.set_index('date')
    df = df.reindex(pd.date_range(df.index.min(), df.index.max(), freq='D').to_period('D').to_timestamp(),
                                  fill_value=0)
    df['date'] = df.index
    df = df.reset_index(drop=True)


    TOOLS = "save, pan, box_zoom, reset, wheel_zoom"

    df_src = ColumnDataSource(df)

    p = figure(plot_height=250, x_axis_type="datetime", 
               tools=TOOLS, toolbar_location=None,
               sizing_mode="scale_width")

    p.add_layout(Title(text = "Number of %s generated every day"%key, text_font_size = "16pt", text_font_style="bold"), 
                 place = 'above')
    p.add_layout(Title(text = "Date Range: %s - %s"%(df['date'].min().strftime('%Y, %b %d'),df['date'].max().strftime('%Y, %b %d'))), 
                 place = 'above')
    p.add_layout(Title(text="Total {} generated: {:,} | Total Days: {:,} ".format(key,df['count'].sum(), len(df)), text_font_style="italic"), 'above')

    p.background_fill_color="#f5f5f5"
    p.grid.grid_line_color="white"
    p.xaxis.axis_label = 'Date'
    p.yaxis.axis_label = 'No. of %s'%key
    p.axis.axis_line_color = None

    p.line(x='date', y='count', line_width=2, color='#ebbd5b', source=df_src)

    p.add_tools(HoverTool(
        tooltips=[( 'date',   '@date{%F}'),
    #               ( 'close',  '$@{adj close}{%0.2f}' ), # use @{ } for field names with spaces
                  ( 'count', '@count'),#{0.00 a}'      ),
                 ],
        formatters={'date'      : 'datetime', # use 'datetime' formatter for 'date' field
    #                 'count' : 'int',   # use 'printf' formatter for 'adj close' field
                                      # use default 'numeral' formatter for other fields
                   },
    #     display a tooltip whenever the cursor is vertically in line with a glyph
    #     mode='vline'
    ))
    show(p)
    save(p, filename='./%s/time_series.html'%key, title='Helioviewer %s generated every day'%key)

# Histogram

In [24]:
def bin_width(m):
    n = np.int(np.log10(m+1))
    n = 10**(n-1)
    q = np.ceil(m/(n*(36))).astype(int)
    bw = max(q*n,1)
    return bw#, m//n+1
for key in hv.keys():
    df = hv[key].copy()
    df = df.set_index('date')
    df = df.reindex(pd.date_range(df.index.min(), df.index.max(), freq='D').to_period('D').to_timestamp(),
                                  fill_value=0)
    df['date'] = df.index
    df = df.reset_index(drop=True)

    name = ''
    name_ = name.replace(" ", "_")

    bin_size = bin_width(df['count'].max())# np.arange(0,count.max(),) 30#.astype(int)#100
    counts = df['count']
    title=name

    arr_hist, edges = np.histogram(counts, bins=np.arange(0, counts.max()+bin_size, bin_size))
    cum_bin_size  = max(bin_size//10,1)
    cum_hist, cum_edges, patches = plt.hist(counts, bins=np.arange(0,counts.max()+cum_bin_size, cum_bin_size), cumulative=True)
    plt.close()

    # Column data source
    df_hist = pd.DataFrame({'count': arr_hist, 'left': edges[:-1], 'right': edges[1:]})
    total = df_hist['count'].sum()
    df_hist['f_count'] = ['%d' % count for count in df_hist['count']]
    df_hist['f_percent'] = ['%.2f%%' %(count/total*100) for count in df_hist['count']]
    df_hist['f_interval'] = ['[%d,%d) ' % (left, right) for left, right in zip(df_hist['left'], df_hist['right'])]
    hist_src = ColumnDataSource(df_hist)

    #cumulative data
    cumulative_data = cum_hist#np.cumsum(arr_hist)
    x_bins = cum_edges[1:]#edges[1:]# np.arange(0, counts.max(), bin_size)[1:]
    df_cum = pd.DataFrame({'count_cum': cumulative_data, 'x': x_bins})
    df_cum['f_percent'] = ['%.2f%%' %(count/total*100) for count in df_cum['count_cum']]
    cum_src = ColumnDataSource(df_cum)

    panels = []
    for axis_type in ["log","linear"]:
        p = figure(y_axis_type = axis_type,
                   x_axis_label = 'No. of %s'%key, y_axis_label = 'Day count', 
                   background_fill_color="#fafafa",
                   y_range = (0.9, df_hist['count'].max() + df_hist['count'].max()//10))

        # Add a quad glyph with source this time
        p.quad(bottom=0.9, top='count', left='left', right='right', source=hist_src, fill_color='navy', alpha=0.5,
               hover_fill_color='navy', hover_fill_alpha=0.2, line_color='white', legend='Histogram')
    #         p.y_range(Range1d(0.8,df_hist['count'].max()))
        # Add style to the plot
        p.title.align = 'center'
        p.title.text_font_size = '18pt'
        p.xaxis.axis_label_text_font_size = '12pt'
        p.xaxis.major_label_text_font_size = '12pt'
        p.yaxis.axis_label_text_font_size = '12pt'
        p.yaxis.major_label_text_font_size = '12pt'
    #         p.add_layout(Span(location=1800, dimension='height'))#, legend='Expected date file count'))
        p.line(x=np.nanmean(counts), y=df_hist['count'], line_color="black", line_dash='solid', line_width = 4, legend="Mean # Data files")
        p.line(x=np.nanmedian(counts), y=df_hist['count'], line_color = "red", line_dash='dashed', line_width=3, legend="Median # Data files")
        p.line(x=stats.mode(counts)[0][0], y=df_hist['count'], line_color = "lightgreen", line_dash = 'dashdot',line_width=2,legend="Mode # Data files")

        total_days = (counts>=0).sum()
        total_files = counts.sum()

        p.add_layout(Title(text = "Histogram for %s generated per day"%key, text_font_size = "16pt", text_font_style="bold"), place = 'above')
        p.add_layout(Title(text="Date range: %s - %s"%(df['date'].min().strftime('%Y, %b %d'),df['date'].max().strftime('%Y, %b %d'))), 'above')
        p.add_layout(Title(text="Total {} generated: {:,} | Total Days: {:,} ".format(key, total_files, total_days), text_font_style="italic"), 'above')

        p.legend.location = "top_right"
    #     p.grid.grid_line_color="white"

    #     text_source = ColumnDataSource(dict(x=[x_bins.max()*3/4],y=[df_hist['count'].max()*3/4],text=['Total Day Count = \n %d'%total]))
    #     glyph = Text(x="x", y="y", text="text", text_color="black")
    #     p.add_glyph(text_source, glyph)

        # Add a hover tool referring to the formatted columns
        hover = HoverTool(tooltips = [('#%s generated'%key, '@f_interval'),
                                      ('Day count', '@f_count'),
                                      ('Day count percentage', '@f_percent')],
                          mode= 'vline')

        # Add the hover tool to the graph
        p.add_tools(hover)
        p2 = figure(y_axis_type=axis_type,
                           x_axis_label = 'No. of %s'%key, 
                           y_axis_label = 'Day count',
                           background_fill_color="#fafafa")


        p2_line = p2.line(x='x', y='count_cum', line_color='#036564', line_width=3, source=cum_src, legend="Cumulative distribution")
    #         p2_circle = p2.circle(x='x', y='count_cum', line_color='#036564', line_width=5, source=cum_src, hover_line_alpha=0.5, legend="Cumulative distribution" )
        p2.add_layout(Title(text = "Cumulative distribution for number of %s generated"%key, text_font_size = "16pt", text_font_style="bold"), place = 'above')
        p2.add_layout(Title(text="Date range: %s - %s"%(df['date'].min().strftime('%Y, %b %d'),df['date'].max().strftime('%Y, %b %d'))), 'above')
        p2.add_layout(Title(text="Total {} generated: {:,} | Total Days: {:,}".format(key, total_files, total_days), text_font_style="italic"), 'above')
    #             p2.add_layout(Title(text="Total Days: %d"%total_days, text_font_style="italic"), 'above')
    #             p2.add_layout(Title(text="Source ID: %d"%sid, text_font_style="italic"), 'above')

        hover = HoverTool(line_policy='nearest', 
                          tooltips = [('#%s generated'%key, '<@x'), 
                                      ('Percentage of %s generated'%key, '<@f_percent'),
                                      ('Cumulative Day count', '@count_cum')],
                          mode='vline')

    #         p2.add_layout(Span(location=1800, dimension='height'))#, legend='Expected date file count'))
        p2.line(x=np.mean(counts), y=df_cum['count_cum'], line_color="black", line_dash='solid', line_width = 4, legend="Mean # Data files")
        p2.line(x=np.median(counts), y=df_cum['count_cum'], line_color = "red", line_dash='dashed', line_width=3, legend="Median # Data files")
        p2.line(x=stats.mode(counts)[0][0], y=df_cum['count_cum'], line_color = "lightgreen", line_dash = 'dashdot',line_width=2,legend="Mode # Data files")

        # Add the hover tool to the graph
        p2.add_tools(hover)
        p2.title.align = 'center'
        p2.title.text_font_size = '18pt'
        p2.xaxis.axis_label_text_font_size = '12pt'
        p2.xaxis.major_label_text_font_size = '12pt'
        p2.yaxis.axis_label_text_font_size = '12pt'
        p2.yaxis.major_label_text_font_size = '12pt'
        p2.legend.location = "bottom_right"


        grid = gridplot([[p, p2]], sizing_mode='stretch_both')# width_policy='max', height_policy='max')#,plot_width=1200, plot_height=1000, sizing_mode='scale_width')#, plot_width=250, plot_height=250)
        panel = Panel(child=grid, title=axis_type)
    #     panel = Panel(child=p, title=axis_type)
        panels.append(panel)
    tabs = Tabs(tabs=panels)
    show(tabs)
    save(tabs, filename='./%s/histogram.html'%key, title='Histogram for Helioviewer %s generated every day'%key)

# Weekday frequency distribution

In [26]:
for key in hv.keys():
    df = hv[key].copy()
    df['weekday'] = df['date'].dt.weekday_name
    weekdays = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    df = df.groupby('weekday').sum().reindex(weekdays)
    df['weekday'] = df.index
    df = df.reset_index(drop=True)
    df

    # Column data source
    df['percent'] = np.float64(["%.2f"%(count/df['count'].sum()*100) for count in df['count']])
    df['percent%'] = df['percent'].astype(str)+"%"
    df['vbar_top'] = df['count'].astype(str) + '\n' + df['percent'].astype(str)+'%'
    df_src = ColumnDataSource(df)
    panels = []
    for axis_type in ["linear","log"]:
        p = figure(x_range = df['weekday'],
                   y_axis_type = axis_type,
                   x_axis_label = 'Weekdays', y_axis_label = 'Day count', 
                   background_fill_color="#fafafa", aspect_ratio=16/9, plot_width=1000)

        # Add a quad glyph with source this time
        p.vbar(x='weekday', top='count', width=0.75, source=df_src, bottom=0.1,
               hover_fill_alpha=0.5, line_color='white', legend="weekday",
               fill_color=factor_cmap('weekday', palette=bp.Spectral7, factors=df['weekday']),
               hover_fill_color=factor_cmap('weekday', palette=bp.Spectral7, factors=df['weekday']), 
              )
        # Add style to the plot
        p.title.align = 'center'
        p.title.text_font_size = '18pt'
        p.xaxis.axis_label_text_font_size = '12pt'
        p.xaxis.major_label_text_font_size = '12pt'
        p.yaxis.axis_label_text_font_size = '12pt'
        p.yaxis.major_label_text_font_size = '12pt'
        p.xgrid.grid_line_color = None
        p.y_range.start = 0.1
        p.y_range.end = df['count'].max()*1.5
        if(axis_type=="log"): p.y_range.end = df['count'].max()**1.5

        p.add_layout(Title(text = "Frequency of %s generated per weekday"%key, text_font_size = "16pt", text_font_style="bold"), place = 'above')
        p.add_layout(Title(text="Date range: %s - %s"%(hv[key]['date'].min().strftime('%Y, %b %d'),hv[key]['date'].max().strftime('%Y, %b %d'))), 'above')
        p.add_layout(Title(text="Total {} generated: {:,} | Total Days: {:,}".format(key,df['count'].sum(), total_days), text_font_style="italic"), 'above')

        p.legend.orientation = "horizontal"
        p.legend.location = "top_center"
        p.grid.grid_line_color="white"
        labels = LabelSet(x='weekday', y='count', text='percent%', level='glyph',
                          x_offset=-30, y_offset=0, source=df_src, render_mode='canvas')
        p.add_layout(labels)

        # Add a hover tool referring to the formatted column

        hover = HoverTool(tooltips = [('%s generated'%key, '@count'),
                                      ('Percentage of %s generated'%key, '@percent%')],
                          mode= 'vline')

    #     Add the hover tool to the graph
        p.add_tools(hover)
        panel = Panel(child=p, title=axis_type)
        panels.append(panel)
    tabs = Tabs(tabs=panels)
    show(tabs)
    save(tabs, filename='./%s/weekday_freq.html'%key, title='Histogram for Helioviewer %s generated every day'%key)

# Weekday frequency against week number

In [28]:
for key in hv.keys():
    df = hv[key].copy()
    df['weekday'] = df['date'].dt.weekday_name
    df['weeknumber'] = ((df['date']-df['date'][0]).dt.days/7).astype(int)
    # df = df.groupby(['weeknumber','weekday']).sum().reset_index()

    weeknumber = np.array(df['weeknumber'].unique()).astype(str)# hv_cov.index.values#.astype(str)
    # weekdays = weekdays# df['weekday'].unique().astype(str) # np.arange(1,32).astype(str)

    colors = bp.Viridis[256]# ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]

    TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

    # output_file('AIA1600_coverage.html')
    panels = []
    for mapper_type, mapper, ticker in zip(["log", "linear"],
                                           [LogColorMapper, LinearColorMapper],
                                           [LogTicker, BasicTicker]):
        p = figure(#x_axis_type='datetime',
                   x_range=weeknumber, y_range=list(reversed(weekdays)),
                   x_axis_location=None, sizing_mode='stretch_both',# width_policy='max', height_policy='max',#, 
    #                plot_width=2000,
                   x_axis_label="Weeks since first data", y_axis_label="Weekday",
                   tools=TOOLS)

        p.rect(x="weeknumber", y="weekday", width=1, height=1,
               source=df,
               fill_color={'field': 'count', 'transform': mapper(palette=colors, low=0.1, high=np.nanmax(df['count']))},
               line_color=None)

        p.add_tools(HoverTool(tooltips=[('Week Number', '@weeknumber'), ('#%s generated'%key, '@count'), ('Date','@date{%F}')],
                    formatters={'date': 'datetime'}
        ))
        xaxis = LinearAxis(ticker=SingleIntervalTicker(interval=7, num_minor_ticks= 1))
        p.add_layout(xaxis, 'above')
        p.xaxis.axis_label = "Weeks since first data"
        # p.grid.grid_line_color = None
        p.axis.axis_line_color = None
        p.axis.major_tick_line_color = None
        p.axis.major_label_text_font_size = "7px"
        p.axis.major_label_standoff = 0
        p.xaxis.major_label_orientation = np.pi / 3
        p.xaxis.axis_label_text_font_size = "12pt"
    #     p.xaxis.major_label_text_color = {'field': 'weeknumber', 'transform': mapper(palette=bp.Spectral6, low=0.1, high=np.nanmax(df['count']))}
        p.yaxis.axis_label_text_font_size = "12pt"
        p.xaxis.visible = True
        p.xgrid.visible = True

        p.xaxis.major_label_text_font_size = "7pt"
        p.yaxis.major_label_text_font_size = "8pt"

        p.add_layout(Title(text = "Weekday frequency against week number", text_font_size = "16pt", text_font_style="bold"), place = 'above')
        p.add_layout(Title(text="Date range: %s - %s"%(hv[key]['date'].min().strftime('%Y, %b %d'),hv[key]['date'].max().strftime('%Y, %b %d'))), 'above')
        p.add_layout(Title(text="Total {} generated: {:,} | Total days: {:,} ".format(key,df['count'].sum(), len(df)), text_font_style="italic"), 'above')



        num_ticks=10
        if (len(df[df['count']>0]['count'].unique()) <= 10):
            num_ticks = len(df[df['count']>0]['count'].unique())
        color_bar = ColorBar(color_mapper = mapper(palette=colors, low=0.1, high=np.nanmax(df['count'])), 
                             major_label_text_font_size="10px",
                             ticker=ticker(desired_num_ticks=num_ticks),
                             formatter=PrintfTickFormatter(format="%d"),
                             label_standoff=6, border_line_color=None, location=(0, 0))
        p.add_layout(color_bar, 'right')
    #             p.width_policy = 'fit'
    #             p.height_policy = 'fit'
        panel = Panel(child=p, title=mapper_type)
        panels.append(panel)
    tabs = Tabs(tabs=panels)
    show(tabs)
    save(tabs, filename='./%s/weeknumber_frequency.html'%key)